In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

In [5]:
%cd /content/gdrive/MyDrive/Colab Notebooks/analysis_edu/05. 데이터분석 및 데이터 전처리

/content/gdrive/MyDrive/Colab Notebooks/analysis_edu/05. 데이터분석 및 데이터 전처리


In [11]:
googleUrl = "../dataset/customerdata.csv"
csDataGitUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/customerdata.csv"

In [12]:
customerData = pd.read_csv(csDataGitUrl)
customerData

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE
0,A13566,4273.900000,3,6,1.679181,Big-Screen-lover
1,A14219,3642.441950,2,4,2.682023,Sleeping-dog
2,A15312,3653.884565,2,5,3.208202,Sleeping-dog
3,A16605,3713.211107,2,6,0.900000,Early-bird
4,B10634,3391.074215,2,4,2.453656,Sleeping-dog
...,...,...,...,...,...,...
145,Z13253,3678.800000,2,5,2.463670,Big-Screen-lover
146,Z13534,3662.437527,3,6,0.900000,Early-bird
147,Z16428,3516.500000,2,5,2.371301,Big-Screen-lover
148,Z16735,3300.100000,2,5,1.704942,Big-Screen-lover
